<a href="https://colab.research.google.com/github/janlucasdeinhard/pyfiles/blob/master/16_PROJECT_SHS/Functional_Dependency_Discovery_SHS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import warnings,os,sys,math,itertools,datetime,time

import pandas as pd

from sklearn.decomposition import PCA

import plotly.express as px

In [9]:
#path_to_data = 'C:/Users/z003mxpm/Desktop/SHS_CKA_RKA_Accounts.xlsx'
path_to_data = 'drive/MyDrive/00_TEMP_DATA/SHS_CKA_RKA_Accounts.xlsx'

In [10]:
bf = pd.read_excel(path_to_data).drop(columns=['Row ID'])

In [ ]:
bf['Identifier'] = bf[['Account Country','OSC Account Number','Account Name (MainAccount)','Account Name']].apply(lambda x: '||'.join(x),axis=1)

In [11]:
transformer = PCA(n_components=2)
pca_res = transformer.fit_transform(bf[[k for k in bf.columns if bf[k].dtype==np.int64]].values)

In [12]:
bf['PCA0'] = pca_res[:,0]
bf['PCA1'] = pca_res[:,1]

In [13]:
transformer.explained_variance_ratio_

array([0.37716925, 0.15208387])

In [14]:
L = []
for k in range(transformer.components_[0].shape[0]):
    L.append([
        bf[[k for k in bf.columns if k.endswith('(Encoded)') and bf[k].dtype==np.int64]].columns[k],
        transformer.components_[0,k],
        transformer.components_[1,k]
    ])
f_dict = pd.DataFrame(L).melt(id_vars=0,value_vars=[1,2]).rename(columns={
    0:'Feature',
    'variable':'Component',
    'value':'Component Value'
})
f_dict['Component'] = f_dict['Component'].astype(str)

In [15]:
px.bar(
    data_frame=f_dict,
    x='Feature',y='Component Value',#title='PCA0'
    color='Component',barmode='group'
)

In [16]:
px.scatter(
    data_frame=bf,
    x = 'PCA0',y='PCA1',
    color='Zone',hover_name='Identifier'
)

In [30]:
df = bf[[k for k in bf.columns if k.endswith(' (Encoded)')]].fillna(-1)

In [64]:
df.min(axis=0)

Bed Size (Encoded)                                  -1.000000e+00
Sales Activity Target (Encoded)                     -1.000000e+00
Revenue Outside Healthcare (GC)+2019 (Encoded)      -4.141000e+06
New Orders Outside Healthcare (GC)+2019 (Encoded)   -1.461882e+07
Revenue Outside Healthcare (GC)+2021 (Encoded)      -5.816158e+06
New Orders Outside Healthcare (GC)+2021 (Encoded)   -2.083515e+07
Revenue Outside Healthcare (GC)+2020 (Encoded)      -5.520539e+06
New Orders Outside Healthcare (GC)+2020 (Encoded)   -1.653547e+07
Account Class In Vitro (DX) (Encoded)                0.000000e+00
Account Class In Vivo (EQ) (Encoded)                 0.000000e+00
Account Local Attr LOV 4 (Encoded)                   0.000000e+00
Account Local Attr LOV 5 (Encoded)                   0.000000e+00
Account Name for Corr. 4 (Encoded)                   0.000000e+00
Account Prefecture Code (Encoded)                    0.000000e+00
Account Province (Encoded)                           0.000000e+00
Account Se

In [54]:
BATCHSIZE = 25

X = df.values
indices = rnd.permutation(X.shape[0])

L = []
for k in range(0,indices.shape[0],BATCHSIZE):
  L.append(X[indices[k:k+BATCHSIZE],:])
X = np.array(L)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [68]:
import torch
from torch import nn
from torch import functional as F
import torch.optim as optim

In [62]:
in_size = X[0].shape[1]
out_size = in_size

In [78]:
class AE(nn.Module):
  def __init__(self):
    super().__init__()
    self.l_in = nn.Linear(in_features=in_size,out_features=32)
    self.l_h0 = nn.Linear(in_features=32,out_features=16)
    self.l_h1 = nn.Linear(in_features=16,out_features=32)
    self.l_out = nn.Linear(in_features=32,out_features=out_size)
  def forward(self,features):
    x = torch.relu(self.l_in(features))
    x = torch.relu(self.l_h0(x))
    x = torch.relu(self.l_h1(x))
    x = torch.relu(self.l_out(x))
    return x

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AE().to(device)

In [107]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)

criterion = nn.MSELoss()

In [108]:
x = X[0]
x_t = torch.from_numpy(x).float()
y_t = model(x_t)

In [109]:
c_loss = criterion(x_t,y_t)

In [114]:
c_loss

tensor(1.6332e+13, grad_fn=<MseLossBackward0>)

In [111]:
optimizer.zero_grad()

In [112]:
c_loss.backward()

In [113]:
optimizer.step()